In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string

In [2]:
# df = pd.read_csv('datasets/biology.csv')
# df.head()

In [3]:
# def tag_in_text(infile):
    
#     df = pd.read_csv('datasets/%s' % infile)
    
#     df['clean content'] = df['content'].apply(cleanhtml)
#     df['clean title'] = df['title'].apply(cleanhtml)
    
#     tags_in_text = []

#     for i in range(len(df.values)):
#         tags = df['tags'][i].split()
#         for j in range(len(tags)):
#             if tags[j] in df['clean content'][i]:
#                 tags_in_text.append(1)
#             elif tags[j] in df['clean title'][i]:
#                 tags_in_text.append(1)
#             else:
#                 tags_in_text.append(0)
#     return np.mean(tags_in_text)

In [4]:
def create_clean_columns(infile):
    df = pd.read_csv('datasets/%s' % infile)
    
    df['clean content'] = df['content'].apply(cleanhtml)
    df['clean title'] = df['title'].apply(cleanhtml)
    return df

In [5]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    cleantext = cleantext.replace('\n', '')
    cleantext = cleantext.translate(string.maketrans("",""), string.punctuation)
    cleantext = cleantext.lower()
    return cleantext

In [6]:
def do_tfidf(df):
    tvec = TfidfVectorizer(stop_words='english')
    tvec.fit(df['clean content'])

    words_df = pd.DataFrame(tvec.transform(df['clean content']).todense(),
                   columns=tvec.get_feature_names())
    return words_df

In [7]:
def get_best_word(df, index):
    row = df.iloc[index, :]
    row_tuple = zip(row.index, row.values)
    return max(row_tuple, key=lambda x: x[1])

In [8]:
def get_list_of_tags(df):
    list_of_tags = []

    for i in range(len(df)):
        list_of_tags.append(get_best_word(df, i))
    better_list_of_tags = []

    for i in range(len(list_of_tags)):
        better_list_of_tags.append(list_of_tags[i][0])
    
    return better_list_of_tags

In [9]:
# def get_best_word(df, index):
#     row = df.iloc[index, :]
#     row_tuple = zip(row.index, row.values)
#     return max(row_tuple, key=lambda x: x[1])

In [37]:
def get_list_of_tags(df):
    list_of_tags = []

#     for i in range(len(df)):
#         list_of_tags.append(get_best_word(df, i))
#     better_list_of_tags = []
    dft = df.transpose()

    for i in range(len(dft)):
#         print dft.iloc[:,i].idxmax()
        list_of_tags.append(dft.iloc[:,i].idxmax())

#     for i in range(len(list_of_tags)):
#         better_list_of_tags.append(list_of_tags[i][0])
    
    return list_of_tags

In [11]:
def create_prediction_column(df, words_df):
    df['predicted_tag'] = get_list_of_tags(words_df)

In [12]:
df = create_clean_columns('robotics.csv')

In [13]:
print df.shape
df.head()

(2771, 6)


,id,title,content,tags,clean content,clean title
0,1,What is the right approach to write the spin c...,<p>Imagine programming a 3 wheel soccer robot....,soccer control,imagine programming a 3 wheel soccer robot wha...,what is the right approach to write the spin c...
1,2,How can I modify a low cost hobby servo to run...,"<p>I've got some hobby servos (<a href=""http:/...",control rcservo,ive got some hobby servos power hd 1501mgs and...,how can i modify a low cost hobby servo to run...
2,3,What useful gaits exist for a six legged robot...,"<p><a href=""http://www.oricomtech.com/projects...",gait walk,httpwwworicomtechcomprojectslegtimehtm lists t...,what useful gaits exist for a six legged robot...
3,4,Good Microcontrollers/SOCs for a Robotics Project,<p>I am looking for a starting point for my pr...,microcontroller arduino raspberry-pi,i am looking for a starting point for my proje...,good microcontrollerssocs for a robotics project
4,5,Nearest-neighbor data structure for non-Euclid...,<p>I'm trying to implement a nearest-neighbor ...,motion-planning rrt,im trying to implement a nearestneighbor struc...,nearestneighbor data structure for noneuclidea...


In [14]:
words_df = do_tfidf(df)
words_df.head()

,00,000,0000,00000,000000,0000000,00000000000000,00000178,000001updatesi,000002fdefine,...,μ4,μg,μt1θ,π2,π20π2π,π2π2,φ1jxτφ,φg,φsetpointφmeasured,φτφ
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
test = words_df.transpose()
test.head()

,0,1,2,3,4,5,6,7,8,9,...,2761,2762,2763,2764,2765,2766,2767,2768,2769,2770
00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
max_test = max(test.iloc[:, 0])
max_test

0.40580085195154675

In [23]:
test.sort_values(0, ascending=False).head(1)

,0,1,2,3,4,5,6,7,8,9,...,2761,2762,2763,2764,2765,2766,2767,2768,2769,2770
addedvalue,0.405801,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
test.iloc[:,0].idxmax()

u'addedvalue'

In [25]:
np.argmax(max_test, axis=1)

ValueError: axis(=1) out of bounds

In [38]:
create_prediction_column(df, words_df)

addedvalue
1501mgs
andripplecan
systems
noneuclidean
platform
designing
tuning
keys
shooting
lbs
revving
pieces
2d
qualitypower
tread
upright
dimensions
battery
components
antenna
astronomically
servo
foolproof
stable
mecanum
driver
gripper
ball
produce
hall
repos
avr
connectors
twowheeled
emergency
notoriously
neighbors
download
weight
realtime
mc
student
kiva
frequency
bearings
nao
casing
decimal
imu
converges
gps
textures
muscle
types
uniform
icp
parameters
phone
orangutan
vision
analog
rpi
motortomotor
batteries
bytes
forcetorque
motors
kit
beacon
waveforms
bigger
apparently
planes
openservo
3axis
36kg
javascript
transformation
goal
line
machines
ipcbridge
legthe
happening
platform
tendon
inability
pen
topic
cable
steer
agent
reinforcement
android
yaw
posture
robotics
commercialized
fast
board
stall
forward
packages
odometry
train
caused
obtain
head
batteries
zeros
motor
movable
foundation
rs232
steps
particles
systems
ros
nxt
piece
integral
video
geared
code
advicestudy
lid
lens
v

IndexError: single positional indexer is out-of-bounds

In [33]:
df.head()

,id,title,content,tags,clean content,clean title,predicted_tag
0,1,What is the right approach to write the spin c...,<p>Imagine programming a 3 wheel soccer robot....,soccer control,imagine programming a 3 wheel soccer robot wha...,what is the right approach to write the spin c...,583
1,2,How can I modify a low cost hobby servo to run...,"<p>I've got some hobby servos (<a href=""http:/...",control rcservo,ive got some hobby servos power hd 1501mgs and...,how can i modify a low cost hobby servo to run...,2137
2,3,What useful gaits exist for a six legged robot...,"<p><a href=""http://www.oricomtech.com/projects...",gait walk,httpwwworicomtechcomprojectslegtimehtm lists t...,what useful gaits exist for a six legged robot...,2722
3,4,Good Microcontrollers/SOCs for a Robotics Project,<p>I am looking for a starting point for my pr...,microcontroller arduino raspberry-pi,i am looking for a starting point for my proje...,good microcontrollerssocs for a robotics project,575
4,5,Nearest-neighbor data structure for non-Euclid...,<p>I'm trying to implement a nearest-neighbor ...,motion-planning rrt,im trying to implement a nearestneighbor struc...,nearestneighbor data structure for noneuclidea...,627


In [17]:
df.shape

(2771, 7)

In [18]:
sub_df = df[['id', 'predicted_tag']]

In [19]:
sub_df.dtypes

id                int64
predicted_tag    object
dtype: object

In [20]:
sub_df.columns = ['id', 'tags']
sub_df.head()

,id,tags
0,1,addedvalue
1,2,1501mgs
2,3,andripplecan
3,4,systems
4,5,noneuclidean


In [56]:
sub_df.to_csv('submission.csv', encoding='utf-8', index=False)